#**Testing Dataset Capstone Project**


Setelah proses klasifikasi dilakukan untuk memprediksi cluster dari data baru (pengguna) berdasarkan pola fitur yang telah terbentuk, tahapan selanjutnya adalah melakukan pengujian model menggunakan dataset testing.

Dataset yang digunakan adalah hasil clustering `Dataset_final_input_user.csv` yang terdiri dari 20.000 data. Dari total tersebut, sebanyak 5.000 data yang sebelumnya telah dipisahkan khusus untuk tahap pengujian untuk mengevaluasi performa model akan digunakan pada tahapan ini.

# **1. Package/Library yang digunakan**

Pada tahapan pertama dari proses testing yakni akan dilakukan import package/library yang diperlukan untuk kemudahan dalam proses testing

Beberapa library utama telah dijelaskan pada notebook sebelumnya, sehingga pada bagian ini hanya dijelaskan library tambahan yang belum dibahas sebelumnya, yaitu:

*   `from tensorflow.keras.models import load_model`
    > Untuk memuat model deep learning yang sudah dilatih dan disimpan sebelumnya dalam format HDF5 atau SavedModel.

*   `from tensorflow.keras.utils import to_categorical`
    > Untuk mengubah label numerik  menjadi format one-hot encoding, yang sering digunakan dalam klasifikasi multi-kelas agar label bisa kompatibel dengan output model neural network.

*   `from sklearn.model_selection import StandardScaler`
    > Untuk menstandarisasi fitur input dengan cara mengurangi nilai rata-rata dan membaginya dengan standar deviasi, sehingga setiap fitur memiliki distribusi dengan rata-rata 0 dan varians 1.

*   `import gdown`
    > Untuk mengunduh file dari Google Drive secara langsung menggunakan URL yang dibagikan, sering dipakai untuk mengambil model atau dataset yang disimpan secara online.

*   `import joblib`
    > Untuk menyimpan dan memuat model, joblib digunakan untuk objek besar.


In [ ]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import gdown
import joblib

# **2. Data Loading dan Model**

Tahapan kedua dari testing adalah memuat dataset serta memuatnya ke dalam Data Frame dan model yang akan digunakan

## Mengunduh Dataset dari Google Drive

In [ ]:
csv_id = '1HbdUKB9qP4On1ETueWmLSwkoHHGKFTto'
csv_url = f'https://drive.google.com/uc?export=download&id={csv_id}'
csv_filename = 'nolabel_datauji_df_final_4_cluster.csv'
gdown.download(csv_url, csv_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1HbdUKB9qP4On1ETueWmLSwkoHHGKFTto
To: /content/nolabel_datauji_df_final_4_cluster.csv
100%|██████████| 62.5k/62.5k [00:00<00:00, 54.5MB/s]


'nolabel_datauji_df_final_4_cluster.csv'

## Memuat ke dalam DataFrame

Setelah data berhasil diunduh, tahapan selanjutnya adalah memuat data ke dalam sebuah DataFrame dan menyimpannya dalam variabe `df`

In [ ]:
df = pd.read_csv('/content/nolabel_datauji_df_final_4_cluster.csv')
df.head(10)

,Unnamed: 0,Pendapatan Bulanan,Total Pengeluaran,Tabungan,Cicilan Per Bulan,Dana Darurat,Total Aset,Total Utang,Target
0,15095,3479135,3366571,383818,722462,472370,12000000,22396351,0
1,15096,5909635,4748150,883466,878886,438586,9000000,7031095,0
2,15097,4690356,4988510,693808,1158535,161281,7500000,35914600,0
3,15098,5777471,4877955,907218,1485468,702520,3000000,77244349,0
4,15099,5212882,3427158,669527,1120757,624172,10500000,63883164,0
5,15100,1000000,2243060,209026,54785,130050,3000000,2739284,3
6,15101,4388363,3209185,466873,779172,343606,3000000,42854503,0
7,15102,5918662,5044962,610191,1345866,388759,12000000,43067721,0
8,15103,6754983,4600228,897483,645954,708072,7500000,3875727,1
9,15104,7184045,5144483,556053,821015,528456,7500000,18883366,0


## Mengunduh Pipeline `preprocessing.pkl` dan model `model_sakoo_ann`

In [ ]:
preprocessing = joblib.load('/content/preprocessing (2).pkl')
model = load_model('/content/model_sakoo_ann (2).keras')

# **3. Data Preprocessing**

Tahapan ketiga dari testing adalah data preprocessing untuk menyiapkan dataset agar siap digunakan untuk analisis dan prediksi.

## Fungsi Rasio Keuangan

Tahapan pertama dari proses data preprocessing adalah membuat fungsi `compute_ratio` digunakan untuk menghitung berbagai rasio keuangan berdasarkan data pengguna. Fungsi ini menghasilkan enam rasio yakni
1. **ExpanseLessThanIncome** = Pengeluaran < Pendapatan
2. **Saving20Percent** = tabungan >= (pendapatan * 0.2)
3. **DebtUnder30PercentIncome** = cicilan <  (pendapatan * 0.3)
4. **EmergencyFundSufficient** =  dana darurat > (pengeluaran * 3)
5. **AssetMoreThanDebt** = total asset > total utang
6. **DebtRatioUnder50Percent** = AssetMoreThanDebt < 0.5

Perhitungan rasio ini membantu memberikan gambaran yang lebih jelas tentang kondisi kesehatan finansial pengguna dan digunakan sebagai fitur dalam model machine learning.

In [ ]:
def compute_ratio(col):
    epsilon = 1e-6
    pendapatan_bulanan = col.iloc[:, 0]
    total_pengeluaran = col.iloc[:, 1]
    tabungan = col.iloc[:, 2]
    cicilan_perbulan = col.iloc[:, 3]
    dana_darurat = col.iloc[:, 4]
    total_aset = col.iloc[:, 5]
    total_utang = col.iloc[:, 6]

    ExpenseToIncomeRatio = total_pengeluaran / (pendapatan_bulanan + epsilon)
    SavingToIncomeRatio = tabungan / (pendapatan_bulanan + epsilon)
    DebtToIncomeRatio = cicilan_perbulan / (pendapatan_bulanan + epsilon)
    EmergencyFundRatio = dana_darurat / (total_pengeluaran + epsilon)
    AssetToDebtRatio = total_aset / (total_utang + epsilon)
    DebtToAssetRatio = total_utang / (total_aset + epsilon)

    return np.stack([
        ExpenseToIncomeRatio,
        SavingToIncomeRatio,
        DebtToIncomeRatio,
        EmergencyFundRatio,
        AssetToDebtRatio,
        DebtToAssetRatio
    ], axis=1)

## Drop Fitur

Tahap selanjutnya dari proses data preprocessing adalah melakukan **drop fitur** pada fitur `Unnamed: 0` da `Target`, karena fitur ini tidak memiliki informasi penting yang dapat digunakan pada proses predikasi. Kemudian dataset yang telah dilakukan drop disimpan dalam `df_simulasi`


In [ ]:
drop_col=['Unnamed: 0','Target']
clean = df.drop(drop_col, axis=1)
df_simulasi = clean.head(10)
df_simulasi

,Pendapatan Bulanan,Total Pengeluaran,Tabungan,Cicilan Per Bulan,Dana Darurat,Total Aset,Total Utang
0,3479135,3366571,383818,722462,472370,12000000,22396351
1,5909635,4748150,883466,878886,438586,9000000,7031095
2,4690356,4988510,693808,1158535,161281,7500000,35914600
3,5777471,4877955,907218,1485468,702520,3000000,77244349
4,5212882,3427158,669527,1120757,624172,10500000,63883164
5,1000000,2243060,209026,54785,130050,3000000,2739284
6,4388363,3209185,466873,779172,343606,3000000,42854503
7,5918662,5044962,610191,1345866,388759,12000000,43067721
8,6754983,4600228,897483,645954,708072,7500000,3875727
9,7184045,5144483,556053,821015,528456,7500000,18883366


# **4. Testing**

Tahapan terakhir adalah testing, yaitu proses melakukan prediksi menggunakan model yang telah dilatih pada dataset.

## Transformasi dan Prediksi

Tahap selanjutnya adalah melakukan transformasi data dalam `df_simulasi` menggunakan pipeline preprocessing yang telah disimpan. Setelah data ditransformasi menjadi `X_user_ratio`, model ANN digunakan untuk memprediksi label yang sesuai. Hasil prediksi berupa probabilitas untuk setiap kelas, kemudian diambil kelas dengan probabilitas tertinggi menggunakan `np.argmax`.


In [ ]:
X_user_ratio = preprocessing.transform(df_simulasi.values)
prediksi = model.predict(X_user_ratio)
predicted_class = np.argmax(prediksi, axis=1)

print("Prediksi kelas:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediksi kelas: [0 0 0 0 0 3 0 0 1 0]


In [ ]:
X_user_ratio

array([[0.23398288, 0.3015996 , 0.63062327, 0.51848635, 0.01914728,
        0.0710975 ],
       [0.14814362, 0.49749961, 0.39488205, 0.32326948, 0.04782124,
        0.0288904 ],
       [0.284132  , 0.4896307 , 0.78801668, 0.07879302, 0.0065495 ,
        0.1847611 ],
       [0.16949911, 0.5351584 , 0.82845802, 0.5335829 , 0.        ,
        1.        ],
       [0.07180283, 0.39219437, 0.65999129, 0.68874757, 0.00483628,
        0.23515065],
       [0.90078762, 0.79518134, 0.01913525, 0.18323139, 0.04069918,
        0.03401928],
       [0.11041485, 0.28194217, 0.51021618, 0.38312563, 0.0012008 ,
        0.55412524],
       [0.17372131, 0.2654766 , 0.70957592, 0.26092401, 0.00923888,
        0.13810015],
       [0.0841268 , 0.41432342, 0.18250189, 0.57390197, 0.073061  ,
        0.01860363],
       [0.10247019, 0.13698818, 0.25712982, 0.36542617, 0.0138062 ,
        0.09643506]])

**Insight**

Model berhasil memprediksi kondisi keuangan berdasarkan data testing dengan distribusi prediksi.Output yang dihasilkan adalah nilai dari masing-masing ratio tersebut yang telah dilakukan transformasi

##  Dengan Data Input Secara Manual

Dilakukan testing model dengan data input secara manual untuk memastikan bahwa pipeline preprocessing dan model ANN yang telah dilatih dapat memproses dan memberikan prediksi terhadap data baru di luar dataset pelatihan.

In [ ]:
user_input = {
    "Pendapatan Bulanan": 7000000,
    "Total Pengeluaran": 8000000,
    "Tabungan": 100000,
    "Cicilan Per Bulan": 1500000,
    "Dana Darurat": 500000,
    "Total Aset": 50000000,
    "Total Utang": 90000000
}

In [ ]:
user_input_df = pd.DataFrame([user_input])
X_user_ratio = preprocessing.transform(user_input_df)
prediksi = model.predict(X_user_ratio)
predicted_class = np.argmax(prediksi, axis=1)

print("Prediksi kelas:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step
Prediksi kelas: [3]


**Insight**

Model berhasil memprediksi kondisi keuangan berdasarkan data inputan diluar dataset dengan distribusi prediksi.